In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re , string , random

In [2]:
data = pd.read_csv('text_emotion.csv')

In [3]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   author     40000 non-null  object
 3   content    40000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [5]:
data.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

In [7]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [8]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [9]:
def get_words_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [10]:
stop_words = stopwords.words('english')

In [11]:
data.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [12]:
neutral_array = [i.split(' ') for i in np.array(data.content[data.sentiment == 'neutral'])]
worry_array =[i.split(' ') for i in  np.array(data.content[data.sentiment == 'worry'])]
happy_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'happiness'])]
sad_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'sadness'])]
love_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'love'])]
surprise_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'surprise'])]
fun_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'fun'])]
relief_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'relief'])]
hate_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'hate'])]
empty_array = [i.split(' ') for i in np.array(data.content[data.sentiment == 'empty'])]
enthu_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'enthusiasm'])]
bore_array = [ i.split(' ') for i in np.array(data.content[data.sentiment == 'boredom'])]
anger_array =[ i.split(' ') for i in np.array(data.content[data.sentiment == 'anger'])]

n_arrays = [neutral_array, worry_array, happy_array, sad_array, love_array, surprise_array, fun_array, relief_array, hate_array, empty_array, enthu_array, anger_array, bore_array]


In [14]:
neutral_dat  = []
worry_dat  = []
happ_dat = []
sad_dat = []
love_dat = []   
surprise_dat = [] 
fun_dat = [] 
relief_dat = [] 
hate_dat = []
empty_dat = []
enth_dat = []
bore_dat= []
anger_dat = []

arrays = [neutral_dat, worry_dat, happ_dat, sad_dat, love_dat, surprise_dat, fun_dat, relief_dat, hate_dat, empty_dat, enth_dat, anger_dat, bore_dat]

In [15]:
for i, k in zip(n_arrays, arrays):
    for j in i:
        j = [x for x in j if x!='']
        k.append(remove_noise(j,stop_words))
        

In [54]:
happy_array[1]

['So',
 'great',
 'to',
 'see',
 'Oin',
 '&amp;',
 'Cynthia.',
 '',
 'So',
 'happy.',
 '',
 'Dinner',
 'was',
 'great,',
 'cute',
 'little',
 'place.',
 '',
 'Too',
 'bad',
 'Oin',
 'got',
 'sick',
 'afterwards.']

In [16]:
happ_dat[1]

['great',
 'see',
 'oin',
 '&amp;',
 'cynthia.',
 'happy.',
 'dinner',
 'great,',
 'cute',
 'little',
 'place.',
 'bad',
 'oin',
 'get',
 'sick',
 'afterwards.']

In [17]:
all_pos_words = get_all_words(neutral_dat)

In [18]:
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[('get', 705), ('go', 660), ("i'm", 468), ('good', 385), ('work', 351), ('like', 339), ('day', 331), ('one', 285), ('think', 264), ('back', 263)]


In [19]:
happy_model = get_words_for_model(happ_dat)
neutral_model = get_words_for_model(neutral_dat)
worry_model = get_words_for_model(worry_dat)
sad_model = get_words_for_model(sad_dat)
love_model = get_words_for_model(love_dat)
surprise_model = get_words_for_model(surprise_dat)
fun_model = get_words_for_model(fun_dat)
relief_model = get_words_for_model(relief_dat)
hate_model = get_words_for_model(hate_dat)
empty_model = get_words_for_model(empty_dat)
enth_model = get_words_for_model(enth_dat)
bore_model = get_words_for_model(bore_dat)
anger_model = get_words_for_model(anger_dat)

In [20]:
happy_dataset = [(tweet_dict, "happy")
                         for tweet_dict in happy_model]
neutral_dataset = [(tweet_dict, "neutral")
                         for tweet_dict in neutral_model]
worry_dataset = [(tweet_dict, "worry")
                         for tweet_dict in worry_model]
sad_dataset = [(tweet_dict, "sad")
                         for tweet_dict in sad_model]
love_dataset = [(tweet_dict, "love")
                         for tweet_dict in love_model]
hate_dataset = [(tweet_dict, "hate")
                         for tweet_dict in hate_model]
surprise_dataset = [(tweet_dict, "surprise")
                         for tweet_dict in surprise_model]
fun_dataset = [(tweet_dict, "fun")
                         for tweet_dict in fun_model]
relief_dataset = [(tweet_dict, "relief")
                         for tweet_dict in relief_model]
empty_dataset = [(tweet_dict, "empty")
                         for tweet_dict in empty_model]
enth_dataset = [(tweet_dict, "enthusiastic")
                         for tweet_dict in enth_model]
bore_dataset = [(tweet_dict, "boredom")
                         for tweet_dict in bore_model]
anger_dataset = [(tweet_dict, "angry")
                         for tweet_dict in anger_model]

In [21]:
dataset = anger_dataset + bore_dataset + enth_dataset + empty_dataset + relief_dataset + fun_dataset + surprise_dataset + hate_dataset + love_dataset + sad_dataset + worry_dataset + neutral_dataset + happy_dataset

In [22]:
random.shuffle(dataset)

In [23]:
len(dataset)

40000

In [78]:
train_data = dataset[:35000]
test_data = dataset[35000:]

In [79]:
classifier = NaiveBayesClassifier.train(train_data)

In [82]:
print("Accuracy is:", classify.accuracy(classifier, test_data))

In [81]:
print(classifier.show_most_informative_features(10))

Most Informative Features
                    bore = True           boredo : happy  =    180.7 : 1.0
                    hate = True             hate : happy  =    153.9 : 1.0
                   annoy = True            angry : neutra =    133.2 : 1.0
                   stuck = True           boredo : neutra =     87.6 : 1.0
                    mrs. = True           boredo : neutra =     81.1 : 1.0
                sooooooo = True           boredo : neutra =     81.1 : 1.0
                     wtf = True           boredo : neutra =     81.1 : 1.0
                 fifteen = True           boredo : neutra =     81.1 : 1.0
                   hell. = True           boredo : neutra =     81.1 : 1.0
                 sunday. = True            angry : neutra =     79.9 : 1.0
None


In [95]:
custom_sentence = "I failed "

In [96]:
custom_tokens = remove_noise(word_tokenize(custom_sentence))

In [97]:
print('Sentence:',custom_sentence,'\nEmotion:', classifier.classify(dict([token, True] for token in custom_tokens)))

Sentence: I failed  
Emotion: sad
